In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import gmsh
import FuncionesMEF3D as M3D

<center><img src="Diseño Arco Recurvo.jpeg"></center>

In [2]:
gmsh.initialize()

In [3]:
gmsh.model.add('Medio Arco Recurvo')

In [4]:
lc = 10
E = 32e6 #Pa???????
nu=0.3
#Defino los puntos iniciales del modelo
# p1 = gmsh.model.geo.addPoint(0, 0, 0, lc) 
p2 = gmsh.model.geo.addPoint(16, 0, 0, lc) 
p3 = gmsh.model.geo.addPoint(14, 10, 0, lc) 
p4 = gmsh.model.geo.addPoint(0, 78, 0, lc) 
p5 = gmsh.model.geo.addPoint(0, 80, 0, lc)
p6 = gmsh.model.geo.addPoint(2, 80, 0, lc)
p7 = gmsh.model.geo.addPoint(2, 78, 0, lc) 
p8 = gmsh.model.geo.addPoint(16, 10, 0, lc)
p9 = gmsh.model.geo.addPoint(18, 0, 0, lc) # optimizar estos puntos para optimizar 
p10 = gmsh.model.geo.addPoint(17, 5, 0, lc)
p11 = gmsh.model.geo.addPoint(19, 5, 0, lc)
#Defino las curvas que conformarán el contorno:
l1 = gmsh.model.geo.add_spline([p2,p10,p3])
l2 = gmsh.model.geo.addLine(p3, p4)
l3 = gmsh.model.geo.addLine(p4, p5)
l4 = gmsh.model.geo.addLine(p5, p6)
l5 = gmsh.model.geo.addLine(p6, p7)
l6 = gmsh.model.geo.addLine(p7, p8)
l7 = gmsh.model.geo.add_spline([p9,p11,p8])
l8 = gmsh.model.geo.addLine(p9, p2)
l9 = gmsh.model.geo.addLine(p3, p8)
l10 = gmsh.model.geo.addLine(p3, p8)
l11 = gmsh.model.geo.addLine(p4, p7) 
#Defino los contornos:
C_Traccionado= gmsh.model.geo.addCurveLoop([l3,l4,l5,l11],reorient=True)
C_Pala= gmsh.model.geo.addCurveLoop([l2,l11,l6,l10],reorient=True)
C_Grap= gmsh.model.geo.addCurveLoop([l1,l9,l7,l8],reorient=True)
#Defino las superficies:
S_Traccionado = gmsh.model.geo.addPlaneSurface([C_Traccionado])
S_Pala = gmsh.model.geo.addPlaneSurface([C_Pala])
S_Grap = gmsh.model.geo.addPlaneSurface([C_Grap])
gmsh.model.geo.synchronize()
#Extrudo el Volumen,para eso debo definir el PG y la Entitie de la superficie:
#gmsh.fltk.run()
V_Traccionado = gmsh.model.geo.extrude([(2,S_Traccionado)],0,0,3)
V_Pala = gmsh.model.geo.extrude([(2,S_Pala)],0,0,3)
V_Grap = gmsh.model.geo.extrude([(2,S_Grap)],0,0,3)
gmsh.model.geo.removeAllDuplicates()
gmsh.model.geo.synchronize()
#gmsh.option.setNumber("Mesh.MeshSizeMin", 1.2)
gmsh.model.mesh.generate(dim=3)
#gmsh.fltk.run()

In [5]:
NodeInfo = gmsh.model.mesh.get_nodes() #saco un objeto con todos los nodos. array uno etiquetas que asigna a los nodos, empieza con 1 y no con 0.
NN = NodeInfo[0].shape[0] #numero de nodos es la cantidad de elementos uqe tengo en el primer array
NN_Tags = NodeInfo[0]-1 #Los Tags de los nodos los pongo en numeración python.
MN = NodeInfo[1].reshape(NN,3)
GLXN=3
E_Tags, MC_Flatten = gmsh.model.mesh.get_elements_by_type(4) #dame los tags (numero de elementos) del tipo 4 (tetrahedros) la MC flatten.
NNXE=4
NE=E_Tags.shape[0]
MC=MC_Flatten.reshape(NE,4)
MC=MC-np.ones(MC.shape)#Lo dejo en numeración python

In [6]:
for e in range(NE):
    Kel, D, B = M3D.k_elemental_3D(MN, MC, nu, E, e)
    K = M3D.Ensamblado_Matriz_Global(MN, MC, Kel, GLXN)

In [7]:
#Busco los tags de la superficie traccionada y del volumen del grap para armar las s.
Tag_S_Traccionada = [20] #la busque en el gmsh
Tag_V_Grap = [tag[1] for tag in V_Grap if tag[0]==3] #me agarro eltag ed dimension 3
#Armo los Physical Groups
Traccionado_PG = gmsh.model.addPhysicalGroup(2, Tag_S_Traccionada) #Tracciono la superficie interna, donde estaría sujetada la cuerda.
Grap_PG = gmsh.model.addPhysicalGroup(3, Tag_V_Grap)
gmsh.model.geo.synchronize()

In [8]:
Traccionado_E=gmsh.model.getEntitiesForPhysicalGroup(2,Traccionado_PG)
ETypesTraccionado, ETagsTraccionado, NodeTagsTraccionado = gmsh.model.mesh.getElements(2,Traccionado_E[0]) 
NElementosTraccionados=len(ETagsTraccionado[0])
MC_T=NodeTagsTraccionado[0].reshape(NElementosTraccionados,GLXN)-1

In [9]:
#Busco los tags del Grap, armo el s y el Us. El grap no se puede mover en ninguna de las 3 direcciones
Tags_Nodos_Grap, CoordFlatten_Nodos_Grap = gmsh.model.mesh.getNodesForPhysicalGroup(3, Grap_PG)
Tags_Nodos_Grap = Tags_Nodos_Grap-1
s=np.array([Tags_Nodos_Grap*GLXN,Tags_Nodos_Grap*GLXN+1,Tags_Nodos_Grap*GLXN+2]).flatten()
Us=np.zeros_like(s)
r=np.array([i for i in range(NN*GLXN) if i not in s]).astype(int)
Fr=np.zeros_like(r).astype(np.float64)
Fx=97.27
Fy=90.7

In [10]:
#poner esquema arco, mostrando angulo, para mi apertura, mi arco es de 30 lbs tensado bla bla bla explicar porque elijo.

In [11]:
for eT in range(NElementosTraccionados):
    n1=MC_T[eT,0].astype(int)
    n2=MC_T[eT,1].astype(int)
    n3=MC_T[eT,2].astype(int)

    A= (1/2)*np.abs(np.linalg.det(np.array([ [MN[n1,1] ,MN[n1,2], 1], 
                                             [MN[n2,1] ,MN[n2,2], 1], 
                                             [MN[n3,1] ,MN[n3,2], 1]])))
    Fr[r==n1*GLXN] += (Fx * A / 3).astype(np.float64)
    Fr[r==n2*GLXN] += (Fx * A / 3).astype(np.float64)
    Fr[r==n3*GLXN] += (Fx * A / 3).astype(np.float64)
    Fr[r==n1*GLXN+1] += (Fy * A / 3).astype(np.float64)
    Fr[r==n2*GLXN+1] += (Fy * A / 3).astype(np.float64)
    Fr[r==n3*GLXN+1] += (Fy * A / 3).astype(np.float64)

In [12]:
U,F = M3D.solve(K, s,r, Us, Fr)

LinAlgError: Singular matrix

In [ ]:
U3D = U.reshape(NN,GLXN)

deformaciones = gmsh.view.add("Desplazamientos")
# por algun motivo le faltaba sumar 1 a nodeinfo
deformaciones_model_data = gmsh.view.addModelData(deformaciones, 0, 'Medio Arco Recurvo', 'NodeData', NodeInfo[0], U3D, numComponents=3)
gmsh.option.setNumber(f'View[{deformaciones}].VectorType',5)

F3D = F.reshape(NN,GLXN)

fuerzas = gmsh.view.add('forces')
# por algun motivo le faltaba sumar 1 a nodeinfo
fuerzas_model_data = gmsh.view.addModelData(fuerzas, 0, 'Medio Arco Recurvo','NodeData',NodeInfo[0], F3D, numComponents=3)
gmsh.option.setNumber(f'View[{fuerzas}].VectorType',4)
gmsh.option.setNumber(f'View[{fuerzas}].GlyphLocation',2)
# gmsh.fltk.run()

gmsh.fltk.run()